In [218]:
import pandas as pd
import os
from termcolor import colored
run_setting_f=open("run_setting_16s_miseq_test.csv")
run_config_dict={}
sample_dict={}
df=pd.DataFrame(columns=['sample_id','concentration','sample count','reads per sample'])

for line in run_setting_f:
    if line.startswith("sample ID"):
        break
    items=line.split(",")
    run_config_dict[items[0]]=items[1]
    
for line in run_setting_f:
    items=line.strip().split(",")
    print (items)

    if items[1]=='':
        df=df.append({'sample_id':items[0],'concentration':items[1],'sample count':float(items[2]),
                      'reads per sample':float(items[3]),'reads required':float(items[2])*float(items[3]),
                     },ignore_index=True)
    else:
        df=df.append({'sample_id':items[0],'concentration':float(items[1]),'sample count':float(items[2]),
                        'reads per sample':float(items[3]),'reads required':float(items[2])*float(items[3]),
                     },ignore_index=True)


total_reads_required=sum(df['reads required'])

df['pct']=df['reads required']/total_reads_required*100
df['pct/con']=df['pct']/df['concentration']
        

file_name='%s_Miseq_%s_%s+%s_%s.xlsx' %(run_config_dict['date'],run_config_dict['nano/micro/v3'],run_config_dict['R1'],
                                   run_config_dict['R2'],run_config_dict['project_id'])




#final loading volume
size=int(run_config_dict['size of library'])
target_cluster_density=int(run_config_dict['target_cluster_density'])

if size<=300:
    loading_pM=target_cluster_density/125
    phix_cluster_efficiency=2.1
elif size<=400:
    loading_pM=target_cluster_density/115
    phix_cluster_efficiency=3
elif size<=500:
    loading_pM=target_cluster_density/105
    phix_cluster_efficiency=3.2
elif size>500:
    loading_pM=target_cluster_density/95
    phix_cluster_efficiency=3.2
    
    

final_loading_20pM_sample=600*float(run_config_dict['% of library'])*loading_pM/20
final_loading_20pM_phix=target_cluster_density*float(run_config_dict['% of phix'])/phix_cluster_efficiency
HT1=600-final_loading_20pM_sample-final_loading_20pM_phix

#to check if the total reads over 15m
if df['reads required'].sum() > 15000000:
    print 
    

run_setting_f.close()    


['1731-474-CL-pool1', '21.17', '30', '50000']
['1731-468-CL-pool1', '0.1', '30', '50000']
['1731-467-plate1-CL-pool1', '1.25', '17', '50000']
['1731-467-plate1-CL-pool2', '15.3', '24', '50000']
['1731-467-plate1-CL-pool3', '131.54', '40', '30000']
['1731-467-plate1-CL-pool4', '71.08', '13', '20000']
['1731-467-plate2-CL-pool1', '22.02', '30', '30000']
['1731-467-plate2-CL-pool2', '350.77', '64', '30000']
['1731-467-plate3-CL-pool1', '17.91', '6', '50000']
['1731-467-plate3-CL-pool2', '41.54', '6', '30000']


In [219]:
df

,sample_id,concentration,sample count,reads per sample,reads required,pct,pct/con
0,1731-474-CL-pool1,21.17,30.0,50000.0,1500000.0,15.290520,0.722273
1,1731-468-CL-pool1,0.10,30.0,50000.0,1500000.0,15.290520,152.905199
2,1731-467-plate1-CL-pool1,1.25,17.0,50000.0,850000.0,8.664628,6.931702
3,1731-467-plate1-CL-pool2,15.30,24.0,50000.0,1200000.0,12.232416,0.799504
4,1731-467-plate1-CL-pool3,131.54,40.0,30000.0,1200000.0,12.232416,0.092994
5,1731-467-plate1-CL-pool4,71.08,13.0,20000.0,260000.0,2.650357,0.037287
6,1731-467-plate2-CL-pool1,22.02,30.0,30000.0,900000.0,9.174312,0.416635
7,1731-467-plate2-CL-pool2,350.77,64.0,30000.0,1920000.0,19.571865,0.055797
8,1731-467-plate3-CL-pool1,17.91,6.0,50000.0,300000.0,3.058104,0.170748
9,1731-467-plate3-CL-pool2,41.54,6.0,30000.0,180000.0,1.834862,0.044171


In [220]:
#divied the sample into different pools
dict_of_df={}

low_pct_con=min(df['pct/con'])
for i in range(1,10):
    if len(df)==0:
        break
    #based on pct/con value, divided into different pool 
    dict_of_df['df_pool%s' %i]=[df[df['pct/con']<=low_pct_con*10].reset_index(drop=True),0]
    #the rest sample will be in another pool
    df=df[df['pct/con']>low_pct_con*10]
    try:
        low_pct_con=min(df['pct/con'])
    except:
        break

for df_name,df_itself_and_water in dict_of_df.items():
    if len(df_itself_and_water[0])>1:
        low_pct_con=10000000000
        total_mol=0
        for i,j in df_itself_and_water[0].sort_values(by='pct/con',ascending=True).iterrows():
            
            low_pct_con=min(low_pct_con,j['pct/con'])
            df_itself_and_water[0].at[i,'add vol']=round(j['pct/con']/low_pct_con,2)
            total_mol+=j['pct/con']/low_pct_con*j['concentration']
            print ('add value = %s' %(j['pct/con']/low_pct_con) )

        #water volumn addded
        water_vol=total_mol/2-df_itself_and_water[0]['add vol'].sum()
        dict_of_df[df_name][1]=water_vol
        

    elif len(df_itself_and_water[0])==1:
        if df_itself_and_water[0]['concentration'][0] > 20:
            df_itself_and_water[0].at[0,'add vol']=2
            water_vol=2*df_itself_and_water[0]['concentration'][0]/2-2
        else:
            df_itself_and_water[0].at[0,'add vol']=20/df_itself_and_water[0]['concentration'][0]
            water_vol=10-20/df_itself_and_water[0]['concentration'][0]
        dict_of_df[df_name][1]=water_vol

add value = 1.0
add value = 1.1846227917484538
add value = 1.49641777101366
add value = 2.494005918059438
add value = 4.579306790362066
add value = 1.0
add value = 1.7335852621634384
add value = 1.9189542483660127


In [221]:
dict_of_df

{'df_pool1': [                  sample_id  concentration  sample count  reads per sample  \
  0  1731-467-plate1-CL-pool3         131.54          40.0           30000.0   
  1  1731-467-plate1-CL-pool4          71.08          13.0           20000.0   
  2  1731-467-plate2-CL-pool2         350.77          64.0           30000.0   
  3  1731-467-plate3-CL-pool1          17.91           6.0           50000.0   
  4  1731-467-plate3-CL-pool2          41.54           6.0           30000.0   
  
     reads required        pct   pct/con  add vol  
  0       1200000.0  12.232416  0.092994     2.49  
  1        260000.0   2.650357  0.037287     1.00  
  2       1920000.0  19.571865  0.055797     1.50  
  3        300000.0   3.058104  0.170748     4.58  
  4        180000.0   1.834862  0.044171     1.18  ,
  516.8823076923077],
 'df_pool2': [                  sample_id  concentration  sample count  reads per sample  \
  0         1731-474-CL-pool1          21.17          30.0           50000.0  

In [222]:

#df_of_pools is used to store the pool id, sum pct of pool, number os samples, and water vol 
df_of_pools=pd.DataFrame(columns=['pool id','sum pct of pool','number of samples'])
for df_name,df_itself_and_water in dict_of_df.items():
    df_of_pools=df_of_pools.append({'pool id':df_name,'sum pct of pool':df_itself_and_water[0]['pct'].sum(),
                                    'number of samples':len(df_itself_and_water[0]),'water vol':df_itself_and_water[1]},ignore_index=True)
    #print (df_name)
    #print ((df_itself_and_water[0]['pct'].sum()))
    #print (len(df_itself_and_water[0]))

#divid 1st pools to different 2nd pools
dict_df_of_pools={}
manual_pool_number=1
for index,item in df_of_pools.iterrows():
    if item['water vol']<0:
        dict_df_of_pools['df_pool_of_pool manual_%s' %manual_pool_number]=df_of_pools.iloc[[index]]
        manual_pool_number+=1
        

df_of_pools=df_of_pools[df_of_pools['water vol']>=0]
low_sum_pct=min(df_of_pools['sum pct of pool'])
for i in range(1,10):
    dict_df_of_pools['df_pool_of_pool %s' %i]=df_of_pools[df_of_pools['sum pct of pool']<low_sum_pct*10].reset_index(drop=True)
    df_of_pools=df_of_pools[df_of_pools['sum pct of pool']>low_sum_pct*10]
    try:
        low_sum_pct=min(df_of_pools['sum pct of pool'])
    except:
        break


In [223]:
dict_df_of_pools

{'df_pool_of_pool manual_1':     pool id  sum pct of pool number of samples  water vol
 2  df_pool3         8.664628                 1       -6.0,
 'df_pool_of_pool manual_2':     pool id  sum pct of pool number of samples  water vol
 3  df_pool4         15.29052                 1     -190.0,
 'df_pool_of_pool 1':     pool id  sum pct of pool number of samples   water vol
 0  df_pool1        39.347604                 5  516.882308
 1  df_pool2        36.697248                 3   39.390000}

In [224]:
for pool_name,df_pool_of_pool in dict_df_of_pools.items():
    df_pool_of_pool['2nd pool add vol']=10*df_pool_of_pool['sum pct of pool']/df_pool_of_pool['sum pct of pool'].sum()
    if min(df_pool_of_pool['2nd pool add vol'])<1:
        df_pool_of_pool['2nd pool add vol']=20*df_pool_of_pool['sum pct of pool']/df_pool_of_pool['sum pct of pool'].sum()
dict_df_of_pools

{'df_pool_of_pool manual_1':     pool id  sum pct of pool number of samples  water vol  2nd pool add vol
 2  df_pool3         8.664628                 1       -6.0              10.0,
 'df_pool_of_pool manual_2':     pool id  sum pct of pool number of samples  water vol  2nd pool add vol
 3  df_pool4         15.29052                 1     -190.0              10.0,
 'df_pool_of_pool 1':     pool id  sum pct of pool number of samples   water vol  2nd pool add vol
 0  df_pool1        39.347604                 5  516.882308          5.174263
 1  df_pool2        36.697248                 3   39.390000          4.825737}

In [225]:
#generate excel
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Alignment
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter

wb_loading=load_workbook("./16s_Miseq_loading_template.xlsx")
ws_loading=wb_loading['loading']

thin = Side(border_style="thin", color="000000")
yellow='00FFFF00'
blue='0000CCFF'
title="Miseq_%s_%s+%s+%s+%s" %(run_config_dict['nano/micro/v3'],run_config_dict['R1'],run_config_dict['I1'],
                              run_config_dict['I2'],run_config_dict['R2'])
ws_loading.cell(row=1,column=1).value=title

row_start=4
row_now=3


for pool_name,df_pool_of_pool in dict_df_of_pools.items():
    second_pool_sum_pct=0
    
    row_2nd_pool_start=row_now+1
    for index,df_pool in df_pool_of_pool.iterrows():
        print (df_pool['pool id'])
        print (df_pool['sum pct of pool'])
        print (df_pool['number of samples'])
        print (df_pool['water vol'])
        
        first_pool_id=df_pool['pool id']
        first_pool_sum_pct=df_pool['sum pct of pool']
        first_pool_sample_number=df_pool['number of samples']
        first_pool_water_vol=df_pool['water vol']
        
        second_pool_sum_pct+=first_pool_sum_pct
        second_pool_add_vol=df_pool['2nd pool add vol']
        
        pool_id=run_config_dict['project_id']+'-loading-'+first_pool_id.split('_')[1]
        ws_loading.cell(row=row_start,column=1).value=pool_id
        
        for index,sample in dict_of_df[first_pool_id][0].iterrows():
            print (index,sample)
            
            row_now+=1   
            ws_loading.cell(row=row_now,column=2).value=sample['sample_id']
            ws_loading.cell(row=row_now,column=3).value=sample['concentration']
            ws_loading.cell(row=row_now,column=4).value=sample['add vol']

        ws_loading.cell(row=row_start,column=5).value=first_pool_water_vol
        ws_loading.cell(row=row_start,column=6).value=second_pool_add_vol
        ws_loading.cell(row=row_start,column=7).value='10ul 2nM sample+10ul 0.1 NaOH+980ul HT1'
        #ws_loading.cell(row=row_start,column=7).value=final_loading_20pM_sample*df_itself['pct'].sum()/100
        
        #merge the cell for different 1st pool
        for col,col_num in zip(['A','E','F'],[1,5,6]):
            print (col,col_num)
            ws_loading.merge_cells("%s%s:%s%s" %(col,row_start,col,row_now))
            ws_loading.cell(row=row_start,column=col_num).alignment=Alignment(horizontal='center',vertical='center')

        row_now+=1
        row_start=row_now+1   
    ws_loading.cell(row=row_2nd_pool_start,column=8).value=final_loading_20pM_sample*second_pool_sum_pct/100
    
    #merge the cell for 20pM sample and pool of 2nM
    
    ws_loading.merge_cells("G%s:G%s" %(row_2nd_pool_start,row_now-1))
    ws_loading.cell(row=row_2nd_pool_start,column=7).alignment=Alignment(horizontal='center',vertical='center')
    ws_loading.merge_cells("H%s:H%s" %(row_2nd_pool_start,row_now-1))
    ws_loading.cell(row=row_2nd_pool_start,column=8).alignment=Alignment(horizontal='center',vertical='center')
    
#merge the cell for final loading part

ws_loading.cell(row=4,column=9).value=final_loading_20pM_phix
ws_loading.cell(row=4,column=10).value=HT1

for col,col_num in zip(['I','J'],range(9,11)):
    print (col,col_num)
    ws_loading.merge_cells("%s4:%s%s" %(col,col,row_now-1))
    ws_loading.cell(row=4,column=col_num).alignment=Alignment(horizontal='center',vertical='center')

    
for row in ws_loading['A%s:I%s' %(1,row_now-1)]:
    for cell in row:
        cell.border=Border(top=thin,left=thin,right=thin,bottom=thin)
for row in ws_loading['D%s:E%s' %(2,row_now-1)]:
    for cell in row:
        cell.fill=PatternFill(start_color=blue,end_color=blue,fill_type='solid')
        
for row in ws_loading['F%s:G%s' %(2,row_now-1)]:
    for cell in row:
        cell.fill=PatternFill(start_color=yellow,end_color=yellow,fill_type='solid')
        
row_now+=2
ws_loading.cell(row=row_now,column=5).value='Box'
ws_loading.cell(row=row_now,column=6).value=run_config_dict['sample pos']

wb_loading.save(file_name)
os.startfile(file_name)

df_pool3
8.664627930682975
1
-6.0
0 sample_id           1731-467-plate1-CL-pool1
concentration                           1.25
sample count                            17.0
reads per sample                     50000.0
reads required                      850000.0
pct                                 8.664628
pct/con                             6.931702
add vol                                 16.0
Name: 0, dtype: object
A 1
E 5
F 6
df_pool4
15.29051987767584
1
-190.0
0 sample_id           1731-468-CL-pool1
concentration                     0.1
sample count                     30.0
reads per sample              50000.0
reads required              1500000.0
pct                          15.29052
pct/con                    152.905199
add vol                         200.0
Name: 0, dtype: object
A 1
E 5
F 6
df_pool1
39.347604485219165
5
516.8823076923077
0 sample_id           1731-467-plate1-CL-pool3
concentration                         131.54
sample count                            40.0
reads p

In [200]:
#generate sample sheet template
sample_sheet_template_f=open('./SampleSheet_miseq_template.csv')
output_sample_sheet=open('%s_%s_SampleSheet.csv' %(run_config_dict['date'],run_config_dict['project_id']),'w+')

for index,line in enumerate(sample_sheet_template_f):
    if index==3:
        output_sample_sheet.write("Experiment-Name,%s\n" %run_config_dict['project_id'] )
    elif index==4:
        output_sample_sheet.write("Date,%s\n" %run_config_dict['date'])
    elif index==12:
        output_sample_sheet.write('%s\n' %(run_config_dict['R1']))
    elif index==13:
        if run_config_dict['R2'] !='0':
            output_sample_sheet.write("%s\n" %run_config_dict['R2'] )
    else:
        output_sample_sheet.write(line)
sample_sheet_template_f.close()
output_sample_sheet.close()

os.startfile('%s_%s_SampleSheet.csv' %(run_config_dict['date'],run_config_dict['project_id']))

In [39]:
df_pool1

NameError: name 'df_pool1' is not defined